# Importing ARCOS Data with Dask

Last week, we used dask to play with a few datasets to get a feel for how dask works. In order to help us develop code that would run quickly, however, we worked with very small, safe datasets. 

Today, we will continue to work with dask, but this time using much larger datasets. This means that (a) doing things incorrectly may lead to your computer crashing (So save all your open files before you start!), and (b) many of the commands you are being asked run will take several minutes each. 

For familiarity, and so you can see what advantages dask can bring to your workflow, today we'll be working with the DEA ARCOS drug shipment database published by the Washington Post! However, to strike a balance between size and speed, we'll be working with a slightly thinned version that has only the last two years of data, instead of all six.

## Exercise 1

Download the thinned ARCOS data [from this link](https://www.dropbox.com/s/o7nc6yvrwog4ozi/arcos_2011_2012.tsv.zip?dl=0). It should be about 2GB zipped, 25 GB unzipped. 

In [1]:
import pandas as pd
import dask.dataframe as dd

pd.set_option("mode.copy_on_write", True)

## Exercise 2

Our goal today is going to be to find the pharmaceutical company that has shipped the most opioids (`MME_Conversion_Factor * CALC_BASE_WT_IN_GM`) in the US.

When working with large datasets, it is good practice to begin by prototyping your code with a subset of your data. So begin by using `pandas` to read in the first 100,000 lines of the ARCOS data and write pandas code to compute the shipments from each shipper (the group that reported the shipment). 

In [2]:
import zipfile
import warnings

warnings.filterwarnings("ignore")

# File path
zip_file_path = "/Users/castnut/Desktop/pds_dask/arcos_2011_2012.tsv.zip"
extracted_folder_path = "/Users/castnut/Desktop/pds_dask/"

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# The path to the extracted file
extracted_file_path = extracted_folder_path + "arcos_2011_2012.tsv"

# Read the first 100,000 lines
df = pd.read_csv(extracted_file_path, sep="\t", nrows=100000)

df["Shipment_Amount"] = df["MME_Conversion_Factor"] * df["CALC_BASE_WT_IN_GM"]

# Group by the shipper and sum the shipment amounts
shipment_totals = df.groupby("Revised_Company_Name")["Shipment_Amount"].sum()

# Find the shipper with the maximum shipment amount
max_shipment_shipper = shipment_totals.idxmax()

print(
    f"The pharmaceutical company that has shipped the most opioids is: {max_shipment_shipper}"
)

The pharmaceutical company that has shipped the most opioids is: Mallinckrodt


In [3]:
# print(df.columns)

In [4]:
# sample = df[["REPORTER_NAME", "Combined_Labeler_Name", "Revised_Company_Name"]].head()
# print(sample)

## Exercise 3

Now let's turn to dask. Re-write your code for dask, and calculate the total shipments by reporting company. Remember: 

- Activate a conda environment with a clean dask installation.
- Start by spinning up a distributed cluster.
- Dask won't read compressed files, so you have to unzip your ARCOS data. 
- Start your cluster in a cell all by itself since you don't want to keep re-running the "start a cluster" code. 

If you need to review dask basic code, [check here](https://nickeubank.github.io/practicaldatascience_book/notebooks/PDS_not_yet_in_coursera/30_big_data/70_dask.html).

As you run your code, make sure to click on the Dashboard link below where you created your cluster:

![dask_dashboard](images/dask_cluster.png)

Among other things, the bar across the bottom should give you a sense of how long your task will take:

![dask_progress](images/dask_progress.png)

(For context, my computer (which has 10 cores) only took a couple seconds. My computer is fast, but most computers should be done within a couple minutes, tops).


In [5]:
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52235,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:52246,Total threads: 2
Dashboard: http://127.0.0.1:52250/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52238,


## Exercise 4

Now let's calculate, *for each state*, what company shipped the most pills?

Note you will quickly find that you can't sort in dask -- sorting in parallel is *really* tricky! So you'll have to work around that. Do what you need to do on the big dataset first, then compute it all so you get it as a regular pandas dataframe, then finish. 

Does this seem like a situation where a single company is responsible for the opioid epidemic?

In [12]:
unziped_file = "arcos_2011_2012.tsv"

dask = dd.read_csv(
    unziped_file,
    sep="\t",
    dtype={
        "REPORTER_STATE": "object",
        "Revised_Company_Name": "object",
        "MME_Conversion_Factor": "object",
        "ACTION_INDICATOR": "object",
        "ORDER_FORM_NO": "object",
        "REPORTER_ADDRESS2": "object",
        "REPORTER_ADDL_CO_INFO": "object",
        "BUYER_ADDL_CO_INFO": "object",
        "BUYER_ADDRESS2": "object",
        "NDC_NO": "object",
        "UNIT": "object",
        "STRENGTH": "float64",
        "BUYER_ZIP": "object",
        "DRUG_CODE": "object",
        "MME_Conversion_Factor": "object",
        "QUANTITY": "object",
        "TRANSACTION_DATE": "float64",
        "TRANSACTION_ID": "float64",
        "dos_str": "object",
    },
)
dask.tail()

,Unnamed: 0,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,...,Product_Name,Ingredient_Name,Measure,MME_Conversion_Factor,Combined_Labeler_Name,Revised_Company_Name,Reporter_family,dos_str,date,year
147655,133669,RA0289062,DISTRIBUTOR,AMERISOURCEBERGEN DRUG CORP,NaN,24903 AVENUE KEARNY,NaN,VALENCIA,CA,91355,...,OXYCODONE HCI 20 MG TABLETS USP,OXYCODONE HYDROCHLORIDE,TAB,1.5,"KVK-Tech, Inc.","KVK-Tech, Inc.",AmerisourceBergen Drug,20.0,2011-01-20,2011
147656,133670,RA0289062,DISTRIBUTOR,AMERISOURCEBERGEN DRUG CORP,NaN,24903 AVENUE KEARNY,NaN,VALENCIA,CA,91355,...,HYDROCODONE BIT. 10MG/ACETAMINOPHEN,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,"Actavis Pharma, Inc.","Allergan, Inc.",AmerisourceBergen Drug,10.0,2011-11-05,2011
147657,133671,RA0289062,DISTRIBUTOR,AMERISOURCEBERGEN DRUG CORP,NaN,24903 AVENUE KEARNY,NaN,VALENCIA,CA,91355,...,HYDROCODONE BIT. 10MG/ACETAMINOPHEN,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,"Actavis Pharma, Inc.","Allergan, Inc.",AmerisourceBergen Drug,10.0,2011-06-08,2011
147658,133672,RA0289062,DISTRIBUTOR,AMERISOURCEBERGEN DRUG CORP,NaN,24903 AVENUE KEARNY,NaN,VALENCIA,CA,91355,...,HYDROCODONE BITARTRATE AND ACETA 7.5,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,"Actavis Pharma, Inc.","Allergan, Inc.",AmerisourceBergen Drug,7.5,2011-11-11,2011
147659,133673,RA0289062,DISTRIBUTOR,AMERISOURCEBERGEN DRUG CORP,NaN,24903 AVENUE KEARNY,NaN,VALENCIA,CA,91355,...,HYDROCODONE BIT. 7.5MG/ACETAMINOPHEN,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,"Actavis Pharma, Inc.","Allergan, Inc.",AmerisourceBergen Drug,7.5,2012-10-22,2012


In [13]:
# Ensure QUANTITY is a numeric type for calculation
dask["QUANTITY"] = dask["QUANTITY"].astype(float)

# Calculate the total pills shipped
dask["Total_Shipments"] = dask["QUANTITY"] * dask["MME_Conversion_Factor"].astype(float)

# Group by state and company, then sum the total shipments
grouped = dask.groupby(["REPORTER_STATE", "Revised_Company_Name"])[
    "Total_Shipments"
].sum()

# Compute to convert to a Pandas DataFrame
grouped_df = grouped.compute()

# Sort within each state group to find the company with the most shipments
result = (
    grouped_df.reset_index()
    .sort_values(["REPORTER_STATE", "Total_Shipments"], ascending=[True, False])
    .groupby("REPORTER_STATE")
    .first()
    .reset_index()
)

result

,REPORTER_STATE,Revised_Company_Name,Total_Shipments
0,AK,Mallinckrodt,57751.0
1,AL,"Endo Pharmaceuticals, Inc.",1216817.0
2,AR,Mallinckrodt,5151362.5
3,AZ,Mallinckrodt,2403539.0
4,CA,"Allergan, Inc.",7543341.0
5,CO,Mallinckrodt,1024245.0
6,CT,"Endo Pharmaceuticals, Inc.",680906.0
7,DC,Mallinckrodt,313.5
8,DE,Mallinckrodt,6.0
9,FL,"Allergan, Inc.",6684920.0


#### *The results  show the company that shipped the most pills in each state.*

## Exercise 5 

Now go ahead and try and re-do the chunking you did by hand for your project (with this 2 years of data) -- calculate, for each year, the total morphine equivalents sent to each county in the US. 

In [19]:
# Ensure QUANTITY and MME_Conversion_Factor are in numeric type for calculation
dask["QUANTITY"] = dask["QUANTITY"].astype(float)
dask["MME_Conversion_Factor"] = dask["MME_Conversion_Factor"].astype(float)

# Calculate the morphine equivalents
dask["Morphine_Equiv"] = dask["QUANTITY"] * dask["MME_Conversion_Factor"]

# Group by year and county, then sum the morphine equivalents
grouped2 = dask.groupby(["year", "BUYER_COUNTY"])["Morphine_Equiv"].sum()

# Compute to convert to a Pandas DataFrame
grouped_df2 = grouped2.compute()

# Pandas DataFrame with total morphine equivalents for each county per year
print(grouped_df2)

year  BUYER_COUNTY
2011  ACADIA           16239.5
      ACCOMACK          7165.0
      ADA             167980.5
      ADAIR            27112.5
      ADAMS           227524.5
                        ...   
      WABAUNSEE            1.0
2012  ARCHER               1.0
      DENALI              10.0
      THROCKMORTON        20.0
      ALPINE               1.0
Name: Morphine_Equiv, Length: 3685, dtype: float64


2023-12-07 17:10:33,081 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 4 memory: 22 MB fds: 33>>
Traceback (most recent call last):
  File "/Users/castnut/Desktop/pds_dask/venv/lib/python3.11/site-packages/tornado/ioloop.py", line 937, in _run
  File "/Users/castnut/Desktop/pds_dask/venv/lib/python3.11/site-packages/distributed/system_monitor.py", line 168, in update
  File "/Users/castnut/Desktop/pds_dask/venv/lib/python3.11/site-packages/psutil/__init__.py", line 2122, in net_io_counters
OSError: [Errno 12] Cannot allocate memory
2023-12-09 17:44:29,237 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 4 memory: 21 MB fds: 32>>
Traceback (most recent call last):
  File "/Users/castnut/Desktop/pds_dask/venv/lib/python3.11/site-packages/tornado/ioloop.py", line 937, in _run
  File "/Users/castnut/Desktop/pds_dask/venv/lib/python3.11/site-packages/distributed/

## Exercise 6

Now, re-write your opioid project's initial opioid import using dask. Each person on your team should create a NEW branch to try this. The person who wrote the initial chunking code can help everyone else understand what they did originally and the data, but everyone should write their own code. 

**WARNING:** You will probably run into a lot of type errors (depending on how the ARCOS data has changed since last year). With real world messy data one of the biggest problems with dask is that it struggles if halfway through dataset it discovers that the column it *thought* was floats contains text. That's why, in the dask reading, [I specified the column type for so many columns](https://nickeubank.github.io/practicaldatascience_book/notebooks/PDS_not_yet_in_coursera/30_big_data/70_dask.html#what-can-dask-do-for-me) as `objects` explicitly. Then, because occasionally there data cleanliness issues, I had to do some converting data types by hand. 

#### *Link for Exercise 6 on GitHub:*
https://github.com/MIDS-at-Duke/opioid-2023-group-8-final-opioid/blob/jiechen_dask/dask_ex6.py 
